In [32]:
#
#  前向传播示例，通过底层实现简单三层网络结构了解深度学习的基本原理
#
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets

# keras自带加载示例数据集的方法，此方法会返回训练数据集和测试数据集
# 这里忽略测试数据集，用x和y分别接收样本和标签
(x, y), _ = datasets.mnist.load_data()
# 默认x和y是numpy类型，转换成tensor，顺便做归一化
x = tf.convert_to_tensor(x, dtype=tf.float32) / 255.
y = tf.convert_to_tensor(y, dtype=tf.int32)

print(x.shape, y.shape)

# 生成训练db，将x和y合并，并定义每个batch的数量
train_db = tf.data.Dataset.from_tensor_slices((x,y)).batch(128)

# 验证一下db的样本是不是我们想要的
train_iter = iter(train_db)
sample = next(train_iter)
print(sample[0].shape, sample[1].shape)

# 初始化参数，我们这次是一个三层网络结构，x的shape是[b, 784]，每层输出的变化如下：
# 第一层：x[b, 784]@w[784, 256] + bias[b] = [b, 256] + [b] = [b, 256]
# 第二层：x[b, 256]@w[256, 128] + bias[b] = [b, 128] + [b] = [b, 128]
# 第三层：x[b, 128]@w[128, 10] + bias[b] = [b, 10] + [b] = [b, 10]
# 参数一定要是tf.Variable类型，才能被记录并计算梯度
w1 = tf.Variable(tf.random.truncated_normal([784, 256], stddev=0.1))  # 一般w通过truncated_normal初始化，标准差设置成0.1，防止梯度爆炸
b1 = tf.Variable(tf.ones([256]))
w2 = tf.Variable(tf.random.truncated_normal([256, 128], stddev=0.1))
b2 = tf.Variable(tf.ones([128]))
w3 = tf.Variable(tf.random.truncated_normal([128, 10], stddev=0.1))
b3 = tf.Variable(tf.ones([10]))
# 学习率
lr = 1e-3

for epoch in range(10):
    
    for step, (x, y) in enumerate(train_db):
        # reshape x
        x = tf.reshape(x, [-1,28*28])
        
        # 需要将网络的前向计算过程包裹在tape中，这样才能记录Variable的变化，从而计算梯度
        with tf.GradientTape() as tape:
            
            # 实际的前向计算过程是：
            # 第一层的输出：h1 = relu(x @ w1 + b1)
            # 第二层的输出：h2 = relu(h1 @ w2 + b2)
            # 第三层的输出：out = h2 @ w3 + b3，最后一层都不需要relu函数
            h1 = x @ w1 + b1
            h1 = tf.nn.relu(h1)

            h2 = h1 @ w2 + b2
            h2 = tf.nn.relu(h2)

            out = h2 @ w3 + b3
            
            # y默认的shape是[b]，而out是[b, 10]
            # 所以y需要做one hot，变成[b, 10]，然后计算loss
            y = tf.one_hot(y, depth=10)
            
            # 计算loss，先对y-out的值求平方
            # 然后求平均，这里的reduce_mean相当于对每个元素求平均值，相当于sum/batch/10，这个对整体loss的梯度并不会产生影响
            mse = tf.square(y - out)
            loss = tf.reduce_mean(mse)
        
        # 通过此方法，求w和b对loss的梯度
        grads = tape.gradient(loss, [w1, b1, w2, b2, w3, b3])
        
        # 原地更新各个参数
        w1.assign_sub(lr*grads[0])
        b1.assign_sub(lr*grads[1])
        w2.assign_sub(lr*grads[2])
        b2.assign_sub(lr*grads[3])
        w3.assign_sub(lr*grads[4])
        b3.assign_sub(lr*grads[5])


        if step%100 == 0:
            print('epoch:', epoch, '\t', 'step:', step, '\t', 'loss:', float(loss))


(60000, 28, 28) (60000,)
(128, 28, 28) (128,)
epoch: 0 	 step: 0 	 loss: 4.965544700622559
epoch: 0 	 step: 100 	 loss: 0.3992805778980255
epoch: 0 	 step: 200 	 loss: 0.3437613546848297
epoch: 0 	 step: 300 	 loss: 0.30093806982040405
epoch: 0 	 step: 400 	 loss: 0.27847203612327576
epoch: 1 	 step: 0 	 loss: 0.23306851089000702
epoch: 1 	 step: 100 	 loss: 0.24529366195201874
epoch: 1 	 step: 200 	 loss: 0.21844303607940674
epoch: 1 	 step: 300 	 loss: 0.2072070837020874
epoch: 1 	 step: 400 	 loss: 0.19642497599124908
epoch: 2 	 step: 0 	 loss: 0.1726073920726776
epoch: 2 	 step: 100 	 loss: 0.1904849112033844
epoch: 2 	 step: 200 	 loss: 0.170650452375412
epoch: 2 	 step: 300 	 loss: 0.1669849157333374
epoch: 2 	 step: 400 	 loss: 0.15884292125701904
epoch: 3 	 step: 0 	 loss: 0.142140731215477
epoch: 3 	 step: 100 	 loss: 0.1613723337650299
epoch: 3 	 step: 200 	 loss: 0.14515100419521332
epoch: 3 	 step: 300 	 loss: 0.14381107687950134
epoch: 3 	 step: 400 	 loss: 0.1370776444673